In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

In [2]:
case = "/glade/scratch/jinmuluo/archive/ColumnTest/lnd/hist/fanv3_improving.clm2.h4.2011-01.nc"
ds = xr.open_dataset(case)

In [3]:
ds

<xarray.Dataset>
Dimensions:                  (levgrnd: 25, levsoi: 20, levlak: 10, levdcmp: 25,
                              time: 1, hist_interval: 2, lon: 144, lat: 96,
                              gridcell: 5662, landunit: 17702, column: 164020,
                              pft: 243288)
Coordinates:
  * levgrnd                  (levgrnd) float32 0.01 0.04 0.09 ... 28.87 42.0
  * levsoi                   (levsoi) float32 0.01 0.04 0.09 ... 5.95 6.94 8.03
  * levlak                   (levlak) float32 0.05 0.6 2.1 ... 25.6 34.33 44.78
  * levdcmp                  (levdcmp) float32 0.01 0.04 0.09 ... 28.87 42.0
  * time                     (time) datetime64[ns] 2011-02-01
  * lon                      (lon) float32 0.0 2.5 5.0 7.5 ... 352.5 355.0 357.5
  * lat                      (lat) float32 -90.0 -88.11 -86.21 ... 88.11 90.0
Dimensions without coordinates: hist_interval, gridcell, landunit, column, pft
Data variables: (12/115)
    mcdate                   (time) int32 ...
    mcsec                    (time) int32 ...
    mdcur                    (time) int32 ...
    mscur                    (time) int32 ...
    nstep                    (time) int32 ...
    time_bounds              (time, hist_interval) datetime64[ns] ...
    ...                       ...
    SMINN_TO_PLANT_FUN_NH4   (time, levdcmp, column) float32 ...
    SMINN_TO_PLANT_FUN_NO3   (time, levdcmp, column) float32 ...
    SMIN_NO3_LEACHED         (time, column) float32 ...
    SMIN_NO3_RUNOFF          (time, column) float32 ...
    SYNTHFERT_N_TO_SMINN     (time, column) float32 ...
    WOOD_HARVESTN            (time, column) float32 ...
Attributes: (12/101)
    title:                                CLM History file information
    comment:                              NOTE: None of the variables are wei...
    Conventions:                          CF-1.0
    history:                              created on 06/20/23 15:36:59
    source:                               Community Terrestrial Systems Model
    hostname:                             cheyenne
    ...                                   ...
    cft_irrigated_tropical_corn:          62
    cft_tropical_soybean:                 63
    cft_irrigated_tropical_soybean:       64
    time_period_freq:                     month_1
    Time_constant_3Dvars_filename:        ./fanv3_improving.clm2.h0.2000-01.nc
    Time_constant_3Dvars:                 ZSOI:DZSOI:WATSAT:SUCSAT:BSW:HKSAT:...

In [4]:
print(ds.NOx_TOTAL.units)

gN/m^2/s


In [22]:
def ds_filter(ds, DataArray, landunit="crop"):
    if landunit == "crop":
        clunit = 2.0
    elif landunit == "natural":
        clunit = 1.0 
    result = DataArray[(ds.cols1d_active.values == 1.0) & (ds.cols1d_itype_lunit.values == clunit)]
    return result

def global_sum(ds, var, landunit="crop"):
    lon_i = ds_filter(ds, ds.cols1d_ixy.values, landunit=landunit).astype(int)-1
    lat_j = ds_filter(ds, ds.cols1d_jxy.values, landunit=landunit).astype(int)-1
    wtgcell = ds_filter(ds, ds.cols1d_wtgcell.values, landunit=landunit)
    val = ds_filter(ds, var, landunit=landunit)
    
    result = 0.0
    # gN/m2 to TgN
    result = result + val * wtgcell * ds.area.values[lat_j, lon_i] *1e-6 *ds.landfrac.values[lat_j, lon_i]
    return sum(result)

In [23]:
result = global_sum(ds, ds.NOx_TOTAL.values[0, :], landunit='crop')
print(result)

2.139927737245039e-08
